In [2]:
from icecube import dataclasses, icetray, dataio
import numpy as np
import matplotlib.pyplot as plt

### Frame structure of IceTray

By default, IceCube software that is base for our development has the following types of frames:
  - Simulation (S)
  - Geometry (G)
  - Calibration (C)
  - DetectorStatus (D)
  - DAQ (Q)
  - Physics (P)

This also defines hierarchy of frames. Typically,content of previous frames of different type can be accessed inside the next frames, unless new frame contains data with the same key, in what case new object is available. 

Special case is made for hierarchy of Q and P frames. When reader finds a new Q frame, the content of P-frame is not available. While subsequent P-frames have access to data of previous Q-frame. The main reason for that is possible frame splitting, when 1 Q frame (corresponding to DAQ reading) can be split into different events, each stored in individual P-frames. Typically frames of higher order are referred as "parent" frame, for example, Q frame is parent of P frame.

Now, let's open an example file and look inside the frames.

In [3]:
infile = dataio.I3File("small_test_file.i3.bz2")

Each file is effectively a stream of individual indendent frames. To get the next frame in the event, one has to call `pop_frame()`, if you are interested in Q or P frame, one can use dedicated `pop_daq()` and `pop_physics()` functions. Important to note that this function gets next frame, making previous frame not accessible (unless it's data from parent frame). Let's get a P-frame and look inside:

In [4]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Geometry):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
]



In [5]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Calibration):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
]



In [6]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Simulation):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [7]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (DAQ):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [8]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [9]:
print(frame)
# hethis one isan example of double stored in P frame
print(frame['PhysParameter'])
# here is a vector of floats 
print(frame[f'DAQObject{frame["PFrameNumber"].value}'])
###

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]

I3Double(507.1)
[197, 117, 932, 706, 861, 199, 109, 753, 229, 700, 384, 238, 579, 670, 673, 369, 825, 563, 325, 142, 456, 853, 223, 49, 570, 768, 969, 740, 615, 842, 118, 188, 875, 140, 587, 752, 931, 787, 623, 746, 124, 215, 840, 996, 408, 239, 264, 121, 714, 570, 777, 149, 556, 622, 779, 80, 673, 98, 754, 185, 682, 364, 17, 509, 730, 408, 391, 669, 957, 534, 582, 434, 476, 38, 508, 958, 837, 654, 680, 167, 11, 552, 194, 476, 375, 951, 180, 550, 684, 122, 56, 410, 712, 205, 279, 737, 882, 978, 843, 351]


Now let's get the next P-frame (number 2) that contains avearage of `DAQObject1`

In [10]:
frame = infile.pop_physics()
print(frame)
# hethis one is an example of double stored in P frame
print(frame['PhysParameter'], frame["PFrameNumber"].value)
# here is a vector of floats 
print(frame[f'DAQObject{frame["PFrameNumber"].value}'])

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]

I3Double(-3913.43) 2
[-4182, -4724, -3246, -3607, -4188, -3404, -3580, -4522, -3939, -4120, -4550, -3683, -4886, -3368, -3100, -4989, -4089, -4540, -3471, -4177, -3804, -3701, -4762, -3441, -4801, -3797, -4972, -3102, -3233, -4719, -3507, -3830, -4256, -4916, -3862, -3256, -4909, -3171, -3263, -3446, -3394, -4754, -3011, -3158, -3590, -4830, -4047, -4424, -4676, -4324, -3552, -4549, -4232, -3269, -3222, -3435, -4878, -3509, -3941, -3507, -3749, -3179, -3233, -3819, -3279, -4126, -3713, -3247, -3153, -3319, -4692, -3681, -4609, -3628, -3342, -3437, -4671, -3469, -4928, -48

As one can see, the frame looks close to a map/dictionary holding serialized data. In printout one can also see the type of the frame which contains certain inforamtio (for example `[DAQ]` or `[Physics]` ). The frame is agnostic to type of data it holds and supports different data types. Most of the available datatypes can be found in dataclasses object (except of `I3Int`, `I3Bool`, which is part of icetray due to historic reasons), plus in individual projects. Standard python or C++ object has to be first converted to icetray object to be stored, as it requires custom serialization. 